In [ ]:
from itertools import product

import torch
import plotly.io as pio
import plotly.express as px
import pandas as pd
import numpy as np
from dask.distributed import Client

from time_series_prediction import settings, utils

pio.templates.default = 'plotly_white'

In [ ]:
# settings.switch_device('cuda')
settings.device == torch.device('cpu')

In [ ]:
params = [
    ('leaking_rate', [0.02, 0.1, 0.3, 0.5, 0.7, 0.9, 1]),
    ('k_l2', 10**torch.linspace(-0.5, -2.5, 5)),
    ('n_neurons', [200, 400, 800, 1000, 1200]),
    ('source', ['vdp', 'rossler', 'lorenz'])
]
# params = [
#     ('k_l2', 10**torch.linspace(-0.5, -2.5, 5)),
#     ('source', ['rossler', 'vdp'])
# ]
keys, vals = zip(*params)
param_dicts = []
for val in product(*vals):
    param_dicts.append({k: v for k, v in zip(keys, val)})
print(f'{len(param_dicts)} simulations')

In [ ]:
n_ensemble = 15

kwargs_default = {
    'spectral_radius': 0.99,
    # 'n_neurons': 200,
}

sweep = utils.SweepESN(
    param_dicts,
    kwargs_default,
    n_ensemble,
    noise=5e-2,
)

In [ ]:
client = Client(n_workers=12, threads_per_worker=1)

In [ ]:
sweep.submit_jobs(client)

In [ ]:
res = sweep.get_results()

In [ ]:
client.shutdown()

In [ ]:
df = pd.DataFrame(res)
df

In [ ]:
df.to_pickle('esn_sweep_lyap_2.pkl')

In [ ]:
df = pd.read_pickle('esn_sweep_lyap_2.pkl')

In [ ]:
for i in range(10):
    err = f'Lyapunov {i}'
    df[f'MSE {err}'] = df['MSE Lyapunov'].apply(lambda x: x[:, i] if x is not None else x)
    df[f'MAE {err}'] = df['MAE Lyapunov'].apply(lambda x: x[:, i] if x is not None else x)
    df[f'Mean MSE {err}'] = df[f'MSE {err}'].apply(lambda x: x.mean() if x is not None else x)
    df[f'Mean MAE {err}'] = df[f'MAE {err}'].apply(lambda x: x.mean() if x is not None else x)
    df[f'Max MSE {err}'] = df[f'MSE {err}'].apply(lambda x: x.max() if x is not None else x)
    df[f'Max MAE {err}'] = df[f'MAE {err}'].apply(lambda x: x.max() if x is not None else x)
    df[f'Min MSE {err}'] = df[f'MSE {err}'].apply(lambda x: x.min() if x is not None else x)
    df[f'Min MAE {err}'] = df[f'MAE {err}'].apply(lambda x: x.min() if x is not None else x)

In [ ]:
row = df.query('source=="rossler" & leaking_rate==0.1 & k_l2==0.01 & n_neurons==1000').sort_values(by='Mean MAE Lyapunov 0').iloc[0]
print(row[[x for x in df.columns if 'MAE' not in x and 'MSE' not in x]])
print(row['Mean MAE Lyapunov 0'])
print(row['Mean MAE Lyapunov 1'])
print(row['Mean MAE Lyapunov 2'])
print(row['Mean MAE Lyapunov 3'])
fig = pio.read_json(row['_fig_path'])
fig.update_layout(
    template='plotly_white', 
    height=800, 
    yaxis1_title_text='x', 
    yaxis2_title_text='y', 
    yaxis3_title_text='z', 
    xaxis5_title_text='t',
)

In [ ]:
row = df.query('source=="vdp" & leaking_rate==0.5 & k_l2==0.1 & n_neurons==400').sort_values(by='Mean MAE').iloc[0]
print(row[[x for x in df.columns if 'MAE' not in x and 'MSE' not in x]])
print(row['Mean MAE Lyapunov 0'])
print(row['Mean MAE Lyapunov 1'])
print(row['Mean MAE Lyapunov 2'])
print(row['Mean MAE Lyapunov 3'])
fig = pio.read_json(row['_fig_path'])
fig.update_layout(
    template='plotly_white', height=800, yaxis_title_text=r'$\frac{dx}{dt}$', yaxis3_title_text='x', xaxis5_title_text='t').write_html(
        'esn_vdp_best.html', include_mathjax='cdn')

In [ ]:
len(list(fig.select_traces({'line_color': 'black', 'legendgroup': 'Test'})))

In [ ]:
from plotly.subplots import make_subplots
from plotly import colors
k_l2_vals = np.sort(df['k_l2'].unique())
c = colors.sample_colorscale(
    colors.get_colorscale('Plasma'),
    np.linspace(0, 0.9, len(k_l2_vals)),
)
n_neuron_vals = df['n_neurons'].unique()
fig = make_subplots(rows=len(n_neuron_vals), cols=1, shared_xaxes='all', shared_yaxes='all')
source = 'lorenz'
dfq = df.query(f'source=="{source}" & leaking_rate==0.1')
for i_neur, n_neur in enumerate(n_neuron_vals):
    for i_l2, k_l2 in enumerate(k_l2_vals):
        df_l2 = dfq[(dfq['k_l2'] == k_l2) & (dfq['n_neurons'] == n_neur)]
        lyap_err = torch.stack(list(df_l2['MAE Lyapunov']))
        t_lyap = torch.zeros_like(lyap_err)
        for i in range(t_lyap.shape[-1]):
            t_lyap[..., i] = i + 1
        fig.add_box(
            x=t_lyap.flatten(),
            y=lyap_err.flatten(),
            name=f'k_l2={k_l2:.4f}',
            row=i_neur+1, col=1,
            legendgroup=f'k_l2={k_l2:.4f}',
            offsetgroup=f'k_l2={k_l2:.4f}',
            marker_color=c[i_l2],
            showlegend=i_neur==0,
            # boxpoints='all'
        )
    fig.update_yaxes(title_text=f'MAE n_neurons={n_neur}', range=[0, 2], row=i_neur+1)
fig.update_xaxes(title_text='Lyapunov time', row=len(n_neuron_vals), dtick=1)
fig.update_layout(boxmode='group', height=1000)

In [ ]:
x = 'leaking_rate'
color = 'n_neurons'
err = ' Lyapunov 0'
y = f'Mean MAE'
max_y = f'Max MAE'
min_y = f'Min MAE'

dfq = df.query('source=="vdp"')

fig = px.line(
    dfq,
    x=x,
    y=y,
    color=color,
    facet_row='k_l2',
    facet_col='source',
    error_y=dfq[max_y] - dfq[y],
    error_y_minus=dfq[y] - dfq[min_y],
    # log_x=True,
    # log_y=True,
    # color_discrete_sequence=px.colors.sample_colorscale(
    #     px.colors.get_colorscale('Plasma'), 
    #     np.linspace(0, 0.9, len(df[color].unique())),
    # )
)
fig.update_traces(mode='markers+lines')
fig.update_layout(height=800)